### Work in progress
Test och se om vi kan bättre utnyttja fält i Litteraturbankens json
* sbl_link --> Wikidata Property 3217 = Svenskt Biografiskt Lexikon
* skbl_link --> Wikidata Property 4963 = Svenskt kvinnobiografiskt lexikon
* sol_link ?!?!?!
* wikidata_id = Wikidata Qnummer
* wikipedia = Wikipedia URL


Denna [Notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/Literaturbanken%20-%20samma%20som.ipynb) 
* [issue 3](https://github.com/salgo60/spa2Commons/issues/3) om vi kan bättre förstå hur Litteraturbanken arbetar

**TODO**
 -  [ ] kontrollera data
 -  [ ] förstå vad SOL är
 - [ ] bättre koll diff SBL SKBL skriva ut till fil
 - [ ] kolla samma som LIBRISXL



**Connectivity status**

I en perfekt värld skall
* A = B = E
* C = F 
* D = G

| Date | A: Litteraturbanken show | B: Littbank - WD | C: Littbank - SBL | D: Littbank - SKBL|E: WD - Littbank | F: WD - Littbank - SBL  |G: WD - Littbank - SKBL  |
| ------------- |:-------------:|:-------------:|:-------------:|:-------------:|:-------------:|:-------------:| -----:|
| 20211001 | 3130 | 2132 | 765 | 152 | 2346 | 810 | 161
| 20211004 | 3130 | 2132 | 765 | 152 | 2347 | 809 | 161
| 20211005 | 3174 | 2354 | 809 | 161 | 2374 | 813 | 163

Saker som detta är enormt viktigt   
* Wikidata behöver auktoriteter att kontrollera kvaliten på vårt data och snabbt se att vi som en öppen plattform inte vandaliseras
* Wikidata med Wikipedia har stort antal visningar - se statistik senaste året > [15000 visningar per dag](https://pageviews.toolforge.org/massviews/?platform=all-access&agent=user&source=category&range=last-year&subjectpage=0&subcategories=0&sort=views&direction=1&view=list&target=https://sv.wikipedia.org/wiki/Kategori:Litteraturbanken) på de artiklar som länkar Litteraturbanken --> vi drar traffik till Litteraturbanken
* vi ser mer och mer brister hos Kungliga biblioteket och andra äldre auktorieteter att dom har gamla plattformar/teknik där fyller en plattform som Wikidata med sina fel och brister en ny möjlighet se [tankar](https://github.com/salgo60/spa2Commons/issues/3) 
* ju fler som kopplar ihop sig med varandra desto snabbare/enklare blir denna process
* på sikt hoppas vi få ett kulturarv där vi jobbar ihop även fast det går långsamt


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-10-05 08:17:31.069650


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None) 
 
url = "https://litteraturbanken.se/api/get_authors"
r = http.request('GET', url)
data = json.loads(r.data)
df = pd.json_normalize(data["data"])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   authorid                       4046 non-null   object 
 1   authorid_norm                  4046 non-null   object 
 2   db_checksum                    4046 non-null   object 
 3   db_timestamp                   4046 non-null   int64  
 4   doc_type                       4046 non-null   object 
 5   full_name                      4046 non-null   object 
 6   gender                         4046 non-null   object 
 7   imported                       3388 non-null   object 
 8   intro                          729 non-null    object 
 9   librisid                       2180 non-null   object 
 10  name_for_index                 4046 non-null   object 
 11  pictureinfo                    169 non-null    object 
 12  searchable                     4046 non-null   b

In [4]:
df["show"].value_counts()

True     3174
False     872
Name: show, dtype: int64

In [5]:
#just objects with show
dfShow = df[df["show"]].copy() 

In [6]:
dfexternal = dfShow[{"authorid","wikidata.sbl_link","wikidata.skbl_link","wikidata.wikidata_id","wikidata.sol_link"
,"wikidata.wikidata_id","wikidata.wikipedia"}]

In [7]:
dfexternal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3174 entries, 1 to 4045
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   wikidata.sol_link     128 non-null    object
 1   authorid              3174 non-null   object
 2   wikidata.wikipedia    1879 non-null   object
 3   wikidata.skbl_link    161 non-null    object
 4   wikidata.wikidata_id  2358 non-null   object
 5   wikidata.sbl_link     810 non-null    object
dtypes: object(6)
memory usage: 173.6+ KB


In [8]:
dfexternal["wikidata.sol_link"].value_counts()

Vilhelm_Fredrik_Palmblad       1
Christopher_Manderström        1
Mårten_Altén                   1
Ulrica_Carolina_Widström       1
Klara_Johanson                 1
                              ..
Ture_Nerman                    1
Ericus_Benedicti_Schroderus    1
Emily_Nonnen                   1
Johan_Henric_Kellgren          1
Wendela_Hebbe                  1
Name: wikidata.sol_link, Length: 128, dtype: int64

In [9]:
dfexternal

,wikidata.sol_link,authorid,wikidata.wikipedia,wikidata.skbl_link,wikidata.wikidata_id,wikidata.sbl_link
1,None,AasenE,None,None,Q11967131,None
2,None,AbeniusM,https://sv.wikipedia.org/wiki/Margit_Abenius,MargitAbenius,Q4933592,None
3,None,AbrahamssonA,https://sv.wikipedia.org/wiki/Augusta_Abrahamsson,None,Q24680938,None
5,None,AbrahamssonS,https://sv.wikipedia.org/wiki/Selma_Abrahamsson,None,Q4934135,None
6,None,AchariusE,https://sv.wikipedia.org/wiki/Erik_Acharius,None,Q365923,5503
...,...,...,...,...,...,...
4037,None,ÖnnerforsA,https://sv.wikipedia.org/wiki/Alf_%C3%96nnerfors,None,Q1237808,None
4039,None,ÖstergrenCL,https://sv.wikipedia.org/wiki/Carl_%C3%96stergren,None,Q6257795,None
4040,None,ÖstergrenC,None,None,Q108406266,None
4043,None,ÖstinO,None,None,Q100752816,None


### Get Wikidata

In [10]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
import pandas as pd 

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# https://w.wiki/4AAV
query = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?WikidataID)  ?authorid ?SBL ?SKBL WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P5101 ?authorid
OPTIONAL {?item wdt:P3217 ?SBL}
OPTIONAL {?item wdt:P4963 ?SKBL}
} order by ?authorid"""
 
    
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDLittbanktot = get_sparql_dataframe(endpoint_url, query)


In [11]:
WDLittbanktot['SBL'] = pd.to_numeric(WDLittbanktot['SBL'], errors="coerce")
WDLittbanktot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2374 entries, 0 to 2373
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WikidataID  2374 non-null   object 
 1   authorid    2374 non-null   object 
 2   SBL         813 non-null    float64
 3   SKBL        163 non-null    object 
dtypes: float64(1), object(3)
memory usage: 74.3+ KB


In [12]:
# Find duplicates
WDLittbanktot[WDLittbanktot.duplicated(["authorid"],keep=False)]

,WikidataID,authorid,SBL,SKBL


In [13]:
dfexternal['wikidata.sbl_link'] = pd.to_numeric(dfexternal['wikidata.sbl_link'], errors="coerce")
dfexternal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3174 entries, 1 to 4045
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   wikidata.sol_link     128 non-null    object 
 1   authorid              3174 non-null   object 
 2   wikidata.wikipedia    1879 non-null   object 
 3   wikidata.skbl_link    161 non-null    object 
 4   wikidata.wikidata_id  2358 non-null   object 
 5   wikidata.sbl_link     810 non-null    float64
dtypes: float64(1), object(5)
memory usage: 173.6+ KB


<ipython-input-13-e1d483007a37>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfexternal['wikidata.sbl_link'] = pd.to_numeric(dfexternal['wikidata.sbl_link'], errors="coerce")


### Compare Littbank Wikidata field with Wikidata 


In [14]:
WDLittbank_WD_merge =  pd.merge(WDLittbanktot, dfexternal, on='authorid',indicator=True)
WDLittbank_WD_merge.rename(columns={"_merge": "WD_Littbank_merge"},inplace = True)

WDLittbank_WD_merge["WD_Littbank_merge"].value_counts()

both          2372
left_only        0
right_only       0
Name: WD_Littbank_merge, dtype: int64

In [15]:
WDLittbank_WD_merge["WD_Littbank_merge"].value_counts()

both          2372
left_only        0
right_only       0
Name: WD_Littbank_merge, dtype: int64

In [16]:
# all with SBL seems to have WIkidata
WDLittbanktotSBL =  WDLittbank_WD_merge[~WDLittbank_WD_merge['wikidata.sbl_link'].isnull()]
WDLittbanktotSBL_noWD = WDLittbanktotSBL[WDLittbanktotSBL['wikidata.wikidata_id'].isnull()] 
WDLittbanktotSBL_noWD

,WikidataID,authorid,SBL,SKBL,wikidata.sol_link,wikidata.wikipedia,wikidata.skbl_link,wikidata.wikidata_id,wikidata.sbl_link,WD_Littbank_merge


### SBL

In [17]:
WDLittbanktot_SBL = WDLittbanktot[~WDLittbanktot["SBL"].isna()]
WDLittbanktot_SBL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 813 entries, 5 to 2303
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WikidataID  813 non-null    object 
 1   authorid    813 non-null    object 
 2   SBL         813 non-null    float64
 3   SKBL        80 non-null     object 
dtypes: float64(1), object(3)
memory usage: 31.8+ KB


In [18]:
# Find duplicates
WDLittbanktot_SBL[WDLittbanktot_SBL.duplicated(["SBL"],keep=False)]

,WikidataID,authorid,SBL,SKBL


In [19]:
dfexternal_SBL = dfexternal[~dfexternal["wikidata.sbl_link"].isna()]

In [20]:
dfexternal_SBL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 810 entries, 6 to 3918
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   wikidata.sol_link     69 non-null     object 
 1   authorid              810 non-null    object 
 2   wikidata.wikipedia    808 non-null    object 
 3   wikidata.skbl_link    79 non-null     object 
 4   wikidata.wikidata_id  810 non-null    object 
 5   wikidata.sbl_link     810 non-null    float64
dtypes: float64(1), object(5)
memory usage: 44.3+ KB


In [21]:
WDLittbank_WD_SBL_merge =  pd.merge(WDLittbanktot_SBL, dfexternal_SBL, left_on='SBL', right_on='wikidata.sbl_link',indicator=True)
WDLittbank_WD_SBL_merge.rename(columns={"_merge": "WD_Littbank_SBL_merge"},inplace = True)

WDLittbank_WD_SBL_merge["WD_Littbank_SBL_merge"].value_counts()

both          810
left_only       0
right_only      0
Name: WD_Littbank_SBL_merge, dtype: int64

## SKBL

In [22]:
WDLittbanktot_SKBL = WDLittbanktot[~WDLittbanktot["SKBL"].isna()]
WDLittbanktot_SKBL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163 entries, 2 to 2363
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WikidataID  163 non-null    object 
 1   authorid    163 non-null    object 
 2   SBL         80 non-null     float64
 3   SKBL        163 non-null    object 
dtypes: float64(1), object(3)
memory usage: 6.4+ KB


In [23]:
dfexternal_SKBL = dfexternal[~dfexternal["wikidata.skbl_link"].isna()]
dfexternal_SKBL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161 entries, 2 to 4028
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   wikidata.sol_link     25 non-null     object 
 1   authorid              161 non-null    object 
 2   wikidata.wikipedia    161 non-null    object 
 3   wikidata.skbl_link    161 non-null    object 
 4   wikidata.wikidata_id  161 non-null    object 
 5   wikidata.sbl_link     79 non-null     float64
dtypes: float64(1), object(5)
memory usage: 8.8+ KB


In [24]:
WDLittbank_WD_SKBL_merge =  pd.merge(WDLittbanktot_SKBL, dfexternal_SBL,how='outer', left_on='SKBL', right_on='wikidata.skbl_link',indicator=True)
WDLittbank_WD_SKBL_merge.rename(columns={"_merge": "WD_Littbank_SKBL_merge"},inplace = True)

WDLittbank_WD_SKBL_merge["WD_Littbank_SKBL_merge"].value_counts()

right_only    731
left_only      84
both           79
Name: WD_Littbank_SKBL_merge, dtype: int64

In [25]:
WDLittbank_WD_SKBL_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 894 entries, 0 to 893
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   WikidataID              163 non-null    object  
 1   authorid_x              163 non-null    object  
 2   SBL                     80 non-null     float64 
 3   SKBL                    163 non-null    object  
 4   wikidata.sol_link       69 non-null     object  
 5   authorid_y              810 non-null    object  
 6   wikidata.wikipedia      808 non-null    object  
 7   wikidata.skbl_link      79 non-null     object  
 8   wikidata.wikidata_id    810 non-null    object  
 9   wikidata.sbl_link       810 non-null    float64 
 10  WD_Littbank_SKBL_merge  894 non-null    category
dtypes: category(1), float64(2), object(8)
memory usage: 77.8+ KB


In [26]:
WDLittbank_WD_SKBL_merge

,WikidataID,authorid_x,SBL,SKBL,wikidata.sol_link,authorid_y,wikidata.wikipedia,wikidata.skbl_link,wikidata.wikidata_id,wikidata.sbl_link,WD_Littbank_SKBL_merge
0,Q4933592,AbeniusM,NaN,MargitAbenius,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Q4933819,AdelborgO,5519.0,OttiliaAdelborg,None,AdelborgO,https://sv.wikipedia.org/wiki/Ottilia_Adelborg,OttiliaAdelborg,Q4933819,5519.0,both
2,Q4346827,AdlersparreS,5564.0,SophieAdlersparre,None,AdlersparreS,https://sv.wikipedia.org/wiki/Sophie_Adlersparre,SophieAdlersparre,Q4346827,5564.0,both
3,Q4933929,AdolfssonE,NaN,EvaAdolfsson,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Q469339,AgrellA,5599.0,AlfhildAgrell,None,AgrellA,https://sv.wikipedia.org/wiki/Alfhild_Agrell,AlfhildAgrell,Q469339,5599.0,both
...,...,...,...,...,...,...,...,...,...,...,...
889,NaN,NaN,NaN,NaN,None,SvedeliusWE,https://sv.wikipedia.org/wiki/Wilhelm_Erik_Sve...,None,Q6197780,34837.0,right_only
890,NaN,NaN,NaN,NaN,None,SwedbergJ,https://sv.wikipedia.org/wiki/Jesper_Swedberg,None,Q748553,34820.0,right_only
891,NaN,NaN,NaN,NaN,None,SwedenborgE,https://sv.wikipedia.org/wiki/Emanuel_Swedenborg,None,Q185832,34840.0,right_only
892,NaN,NaN,NaN,NaN,None,SwederusG,https://sv.wikipedia.org/wiki/Georg_Swederus,None,Q22110411,34844.0,right_only


In [27]:
print("|",start_time.strftime("%Y%m%d"),"|",dfShow["authorid"].nunique(),
     "|",dfShow["wikidata.wikidata_id"].nunique(),
     "|",dfShow["wikidata.sbl_link"].nunique(),
     "|",dfShow["wikidata.skbl_link"].nunique(),
     "|",WDLittbanktot["authorid"].nunique(),
     "|",WDLittbanktot["SBL"].nunique(),
     "|",WDLittbanktot["SKBL"].nunique()
     )

| 20211005 | 3174 | 2354 | 809 | 161 | 2374 | 813 | 163


In [28]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2021-10-05 08:17:37.780049
Time elapsed (hh:mm:ss.ms) 0:00:06.710900


In [29]:
#df[df.duplicated(['ID'], keep=False)]

In [30]:
WDLittbanktot[WDLittbanktot.duplicated(["authorid"],keep=False)]

,WikidataID,authorid,SBL,SKBL


In [31]:
#WDLittbanktot[WDLittbanktot["authorid"]==WDLittbanktot[WDLittbanktot["authorid"].duplicated()]].sort_values("authorid")

In [32]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ass.ms) {}'.format(datetime.now() - start_time))

Ended:  2021-10-05 08:17:37.820705
Time elapsed (hh:mm:ass.ms) 0:00:06.751980
